In [1]:
import pandas as pd
import numpy as np

In [2]:
d = pd.read_csv('ph.csv', compression='gzip')

/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
d.head()

,id,creation_date,post_id,post_history_type_id,revision_guid,user_id,text,comment
0,28369829,2012-08-24 08:18:33.29 UTC,12105586,1,95c45c9a-aff1-4b7c-adc2-76c727c6a930,1165423.0,Loading String Resources from A Different Path,NaN
1,95514764,2015-07-20 18:28:11.287 UTC,31523754,1,c8ef2715-59bb-429b-ad39-25086ec4e123,5136201.0,System.Data.SqlClient.SqlException: Incorrect ...,NaN
2,187966461,2018-12-17 09:16:42.123 UTC,53812067,1,72e5ae09-4edc-44f2-ae53-8eacda39a9f4,2105218.0,Do multiple jdbc input run simultaneously or s...,NaN
3,8155373,2010-10-07 21:33:26.907 UTC,3886039,1,f419429b-9756-43ff-baf4-5927f589c73c,56621.0,"""Test with debugger"" in ReSharper?",NaN
4,40793970,2013-05-17 16:18:08.577 UTC,16613724,1,ef698524-da6f-4a1f-b0ba-310b38e730d7,2394636.0,jQuerymobile: Close a popup when opening a panel,NaN


In [71]:
# demo from online:
import string
import math

tokenize = lambda doc: doc.lower().split(" ")

document_0 = "China has a strong economy that is growing at a rapid pace. However politically it differs greatly from the US Economy."
document_1 = "At last, China seems serious about confronting an endemic problem: domestic violence and corruption."
document_2 = "Japan's prime minister, Shinzo Abe, is working towards healing the economic turmoil in his own country for his view on the future of his people."
document_3 = "Vladimir Putin is working hard to fix the economy in Russia as the Ruble has tumbled."
document_4 = "What's the future of Abenomics? We asked Shinzo Abe for his views"
document_5 = "Obama has eased sanctions on Cuba while accelerating those against the Russian Economy, even as the Ruble's value falls almost daily."
document_6 = "Vladimir Putin was found to be riding a horse, again, without a shirt on while hunting deer. Vladimir Putin always seems so serious about things - even riding horses."

all_documents = [document_0, document_1, document_2, document_3, document_4, document_5, document_6]

tokenized_documents = [tokenize(d) for d in all_documents] # tokenized docs
all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
print(tokenized_documents)

[['china', 'has', 'a', 'strong', 'economy', 'that', 'is', 'growing', 'at', 'a', 'rapid', 'pace.', 'however', 'politically', 'it', 'differs', 'greatly', 'from', 'the', 'us', 'economy.'], ['at', 'last,', 'china', 'seems', 'serious', 'about', 'confronting', 'an', 'endemic', 'problem:', 'domestic', 'violence', 'and', 'corruption.'], ["japan's", 'prime', 'minister,', 'shinzo', 'abe,', 'is', 'working', 'towards', 'healing', 'the', 'economic', 'turmoil', 'in', 'his', 'own', 'country', 'for', 'his', 'view', 'on', 'the', 'future', 'of', 'his', 'people.'], ['vladimir', 'putin', 'is', 'working', 'hard', 'to', 'fix', 'the', 'economy', 'in', 'russia', 'as', 'the', 'ruble', 'has', 'tumbled.'], ["what's", 'the', 'future', 'of', 'abenomics?', 'we', 'asked', 'shinzo', 'abe', 'for', 'his', 'views'], ['obama', 'has', 'eased', 'sanctions', 'on', 'cuba', 'while', 'accelerating', 'those', 'against', 'the', 'russian', 'economy,', 'even', 'as', 'the', "ruble's", 'value', 'falls', 'almost', 'daily.'], ['vladim

In [72]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [73]:
%timeit a = jaccard_similarity(tokenized_documents[2],tokenized_documents[4])

4.22 µs ± 89.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [74]:
# term-frequency inverse document frequency
def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

def sublinear_term_frequency(term, tokenized_document):
#     print(tokenized_document.count(term))
#     print(tokenized_document, term)
    try:
        return 1 + math.log(tokenized_document.count(term))
    except ValueError:
        return 0
    

def augmented_term_frequency(term, tokenized_document):
    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))

def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

In [75]:
idf_values = inverse_document_frequencies(tokenized_documents)
print(idf_values['abenomics?'])
# 2.94591014906
print(idf_values['the'])

2.9459101490553135
1.336472236621213


In [76]:
def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

In [77]:
%time tfidf_representation = tfidf(all_documents)
# print(tfidf_representation[0], document_0)
# doc vector and document

CPU times: user 1.55 ms, sys: 0 ns, total: 1.55 ms
Wall time: 1.56 ms


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)

sklearn_representation = sklearn_tfidf.fit_transform(all_documents)
print(tfidf_representation[0])
print(sklearn_representation.toarray()[0].tolist())
print(document_0)

[2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 1.336472236621213, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 3.8142592685777856, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 2.9459101490553135, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8472978603872037, 2.9459101490553135, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 1.8472978603872037, 0.0, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.31019096605521496, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18320378146489946, 0.0, 0.18320378146489946, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23957330918096045, 0.0, 0.0, 0.0, 0.183203

In [49]:
d['text'][1]

"System.Data.SqlClient.SqlException: Incorrect syntax near the keyword 'FROM'"

In [64]:
import pandas as pd
d2 = pd.read_csv('posts_questions.csv', compression='gzip')

In [65]:
def clean(line):
#     print(type(line))
    line = line.lower().replace("\n"," ").replace("\r","").replace(',',"")
    return line
# clean(d2['body'][0])
# d2.to_csv('pq2.csv', sep='~')
col = []
for text in d2['body']:
    col.append(clean(text))
d2['cleaned'] = col
print(d2['cleaned'][0])
d2['cleaned'].to_csv('pq_clean.csv')
# print(d2['body'][0])

<p>i am trying to create a report to display a summary of the values of the columns for each row.   a basic analogy would an inventory listing.  say i have about 15 locations like 2a 2b 2c 3a 3b 3c etc.   each location has a variety of items and the items each have a specific set of common descriptions i.e. a rating of 1-9 boolean y or n another boolean y or n.  it looks something like this:</p>  <pre><code>2a   4       y       n 2a   5       y       y 2a   5       n       y 2a   6       n       n       ... 2b   4       n       y   2b   4       y       y       ...etc. </code></pre>  <p>what i would like to produce is a list of locations and summary counts of each attribute:</p>  <pre><code>location    1 2 3 4 5 6 7 8 9      y  n        y n      total 2a                1 2 1            2  2        2 2        4 2b                2                1  1        2          2 ... ___________________________________________________________ totals            3 2 1            3  3        4 2     

/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if sys.path[0] == '':


In [1]:
# d.head()
# d = 5
def word_count(row):
    return len(row.split(" "))
#     print(row)
#     l = row.split(',')[2]
#     print(l)
#     return len(l.split(" "))

from pyspark import SparkContext
def process(filename):
    sc = SparkContext(appName="Text Process").getOrCreate()
    data = sc.textFile(filename)
#     sc.addPyFile("mfun.pyx")
    scores = sc.parallelize(data).map(mfun.word_count).collect()
    sc.stop()
    return scores

In [88]:
# print(d2[:10])
# sc.stop()
# d3 = d2[:100]
# d3.to_csv('pq.csv')
# sc = SparkContext(appName = 'TextProcess').getOrCreate()
# def line(a):
#     sp = a.split(",")
#     return a.split("~")
# process('pq.csv')
def word_count(row):
    return len(row.split(" "))

files = sc.textFile("pq_clean.csv") \
.map(lambda line: line.split(",")[1]) \
.cache()
# .map(word_count) \
# .cache()
# .sum()
#     .map(lambda line: line.split(",")) \
#     .collect()

In [91]:
from pyspark.mllib.feature import HashingTF, IDF

hashingTF = HashingTF()
tf = hashingTF.transform(files)
tf.cache()  

PythonRDD[104] at RDD at PythonRDD.scala:53

In [ ]:
# tf.take(1)
idf = IDF().fit(tf)
%time tfidf = idf.transform(tf)

In [84]:
# df = pd.read_csv("pq_clean.csv")
df.head()

,0,<p>i am trying to create a report to display a summary of the values of the columns for each row. a basic analogy would an inventory listing. say i have about 15 locations like 2a 2b 2c 3a 3b 3c etc. each location has a variety of items and the items each have a specific set of common descriptions i.e. a rating of 1-9 boolean y or n another boolean y or n. it looks something like this:</p> <pre><code>2a 4 y n 2a 5 y y 2a 5 n y 2a 6 n n ... 2b 4 n y 2b 4 y y ...etc. </code></pre> <p>what i would like to produce is a list of locations and summary counts of each attribute:</p> <pre><code>location 1 2 3 4 5 6 7 8 9 y n y n total 2a 1 2 1 2 2 2 2 4 2b 2 1 1 2 2 ... ___________________________________________________________ totals 3 2 1 3 3 4 2 6 </code></pre> <p>the query returns fields: </p> <pre><code>location_cd string desc_cd int y_n_1 string y_n_2 string </code></pre> <p>i have tried grouping by location but cannot get the summaries to work. i tried putting it in a table but that would only take the original query. i tried to create datasets for each unit and create variables in each one for each of the criteria but that hasn't worked yet either. but maybe i am way off track and crosstabs would work better? i tried that and got a total mess the first time. maybe a bunch of subreports?</p> <p>can someone point me in the correct direction please? it seemed easy when i started out but now i am getting nowhere. i can get the report to print out the raw data but all i need are totals for each column broken down out by location. </p>
0,1,<p>i understand payments are a tricky thing bu...
1,2,<p>i am making a class to store a reference to...
2,3,<p>i am trying to compare performance of <code...
3,4,"<p>when i am reading the code of <a href=""http..."
4,5,<p>i am trying to work why logging out the con...


In [78]:
%time tfidf_representation = tfidf(df)

CPU times: user 4.29 ms, sys: 12 µs, total: 4.31 ms
Wall time: 4.22 ms


In [90]:
# print(tfidf_representation[1])

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
%time sklearn_representation = sklearn_tfidf.fit_transform(df)

CPU times: user 6.24 ms, sys: 102 µs, total: 6.34 ms
Wall time: 6.68 ms


In [89]:
# print(len(df))
# print(sklearn_representation[1])